In [ ]:
rm(list = ls())         # Remove all objects from the workspace
graphics.off()          # Close all open graphics devices

# Load required libraries
library(settings)       # For resetting global options
reset(options)          # Reset all global options to default values

library(nlmeU)          # For datasets
library(nlme)           # For model implementation
library(lattice)        # For lattice plotting
library(corrplot)       # For correlation matrix visualization
library(plot.matrix)    # For matrix plots
library(MASS)           # For statistical functions
library(car)            # For regression diagnostics
library(lme4)           # For mixed-effects models
library(insight)        # For model insight extraction
library(rgeoda)         # For geospatial analysis
library(sf)             # For handling spatial data
library(sp)             # For spatial data management
library(gstat)          # For geostatistical analysis
library(geoR)           # For geostatistical modeling
library(geojsonio)
library(KFAS)
library(HMM)

set.seed(????)

# Set working directory
setwd("/Users/amirh_jandaghian/Documents/")

In [ ]:
data  <- read.table('data.text')

In [ ]:
####### Visualization #######

### line chart for each section
xy1 <- xyplot(Y ~ X | section.f,
    # visual and time are plotted against each other in separate
    # panels for different values of 'treat.f' factor.
    groups = grouping_col,
    data = data,
    type = "l",
    lty = 1
)

### boxplot for each section
bw1 <- bwplot(y ~ x.f | sectuib.f, data = data)  # bwplot from package lattice
    #xlims <- c("Base", "4\nwks", "12\nwks", "24\nwks", "52\nwks")
    #update(bw1, xlim = xlims, pch = "|" )

#### Linear Models

In [ ]:
#### Fitting and Results - homo

#(GLS)
lm1.form <- visual ~ -1 + visual0 + time.f + treat.f:time.f   # model defined last time
fm6.1 <- gls(lm1.form, data = armd)
summary(fm6.1)


#---------------------------------------------------------------------------------#---------------------------------------------------------------------------------
# MODEL FORMULATION in R: (LM)
form.form <- formula(   visual ~ -1 + visual0 + time.f + treat.f:time.f    ) #forms on the 1.2 file
# we now fit the linear model
lm_fit <- lm(    form.form, data = armd   )  

sum = summary(lm_fit) 
summary(lm_fit) 

### GLS fitting in the file 1.1 

#---------------------------------------------------------------------------------
# How to obtain information from our lm
lm_fit$coefficients       # the betas
#lm_fit$residuals          # ^epsilon
#lm_fit$fitted.values      # ^y
lm_fit$rank               # rank
lm_fit$df.residual        # degrees of freedom of the model (n-rank)
#lm_fit$model              # the dataframe
#rstandard(lm_fit)         # the standardized residuals
summ$sigma               # sigma (estimate of the residual standard deviation)

vcov(lm_fit)              # variance-covariance matrix
sqrt(diag(vcov(lm_fit)))  # standard errors of the coefficients


#---------------------------------------------------------------------------------
# Confidence intervals for the coefficients
alpha = 0.05
confint(lm_fit,level = 1 - alpha)

# We can correct them through Bonferroni
confint(lm_fit,level = 1 - alpha/length(lm_fit$coefficients))

In [ ]:
### Predicting

# Step 1: we generate new observations - 3 patients (within the range of the observed values)
X.new <- data.frame(time.f = ordered(rep(c('4wks', '12wks', '24wks', '52wks' ),3)),
                visual0 = c(rep(50,4), rep(60,4), rep(55,4)),
                treat.f = factor(c(rep('Active',4), rep('Placebo',4), rep('Active',4)))
)

X.new <- data.frame(colname1 = c(11,11,11)^2, name2=c(1,0,0), name3=c(0,1,0))

X.new

# Step 2: we build the intervals (for each single observation)
IC <-predict(lm_fit, X.new, interval="confidence", level=0.95)
IC 

IP <-predict(lm_fit, X.new, interval="prediction", level=0.95)
IP 

#?predict.lm # look at the plot in the example below

In [ ]:
### Evaluating the model

anova(lm6.1_2, lm6.1)                         # fm6.1_2 nested in fm6.1
# We look at the AIC because the models are non-nested and LR test cannot be performed

AIC(lm6.1_2, lm6.1)                           # AIC of the two models

In [ ]:
#### Diagnostics plots
par(bg = "white") # Sets the background color to white

# Automatic diagnostic plots (graphically)
#x11()
par(mfrow = c(1,1))
plot(lm_fit) #resid vs fitted values 

shapiro.test(lm_fit$residuals) # problem confirmed by shapiro.test
#If the p-value is small (typically < 0.05), it suggests the residuals are NOT normally distributed.

plot(lm_fit$residuals) # resid vs index
abline(h=0)




# color - let's color the residuals relative to different columns
x = data$column
par(bg = "white") # Sets the background color to white
colori = rainbow(length(unique(x))) # Creates a color palette for each unique value in 'x'
num_sub = table(x) # Gets the frequency of each subject
colori2 = rep(colori, num_sub) # Replicates the colors based on frequency
plot(lm_fit$residuals, col=colori2) # Plots residuals with respective colors
abline(h=0) # Adds a horizontal line at y=0




# boxplot of residuals
x = data$column.f
boxplot(lm_fit$residuals ~ x, col=colori,
        xlab='Time.f', ylab='Residuals') # --> informative
# -> the variance of the observations increases in time





#GLS models lab2
plot(fm9.2, resid(., type = "response") ~ time)       # Raw residuals vs time 
bwplot(resid(fm9.2, type='response') ~ time.f,        # Raw residuals vs time.f
        pch = "|", data = armd)      

# Pearson residuals [ ^eps_i/sqrt(Var(y_i)) ]
plot(fm9.2, resid(., type = "pearson" ) ~ fitted(.)) # Pearson vs fitted
plot(fm9.2, resid(., type = "pearson") ~ time)       # Pearson vs time 
bwplot(resid(fm9.2, type = "pearson") ~ time.f,pch = "|", data = armd)     # Pearson vs time.f

In [ ]:
#### Fitting and Results -  independent, heteroscedastic residual errors 

#_varClass___________parameters___________________Group #########################################
# varFixed()         value                        known weights
# varIdent()         value, form, fixed           <delta>-group
# varExp()           value, form, fixed           <delta>-group, <delta,mu>-group, <mu>-group
# varPower()         value, form, fixed           <delta>-group, <delta,mu>-group, <mu>-group
# varConstPower()    const, power, form, fixed    <delta>-group, <delta,mu>-group, <mu>-group


###### model 9.0: lambda_i are known, i.e. lambda(v) ######
fm9.0 <- gls(visual ~ -1 + visual0 + time.f + treat.f:time.f, 
             method = 'REML',                   # default
             weights = varFixed(value = ~time), # Var.function; lambda_i(v_i) (v_i is known and observable)
             data = armd)


###### model 9.1: <delta>-group i.e. lambda(delta,v) ######
fm9.1 <- gls(visual ~ -1 + visual0 + time.f + treat.f:time.f,                  
         weights = varIdent(form = ~ 1|time.f), # Var.function; <delta>-group i.e. lambda(delta,v)
                    # delta_1 = 1
                    # delta_2 = sigma_2/sigma_1
                    # delta_3 = sigma_3/sigma_1
                    # delta_4 = sigma_4/sigma_1
         data = armd)
# form ~ v, or ~ v | g, specifying a variance covariate v and, optionally, a grouping factor g for the coefficients.
# The variance covariate is ignored in this variance function.

# sigma_it = sigma * lambda_it 
#          = sigma * lambda(delta, TIME_it) 
#          = sigma * |TIME_it|^delta            
weights = varPower(form = ~time)

# sigma_it = sigma * lambda_it 
#          = sigma * lambda(delta, TIME_it) 
#          = {sigma * |TIME_it|^delta_1    ACTIVE
#            {sigma * |TIME_it|^delta_2    PLACEBO
weights = varPower(form = ~time|treat.f) #strata=treat.f  

fm9.2$modelStruct$varStruct                          # our delta is estimated      


###### model 9.4 (REML-based GLS) <delta,mu>-group i.e. lambda(delta,mu,v) ######
# delta = scalar, no strata 
# sigma_it = sigma * lambda_it 
#          = sigma * lambda(delta, mu_it) 
#          = sigma * |mu_it|^delta      where mu_it = b_0t + b1 * VISUAL0_i + b_2t * TREAT_i
#                                             predicted mean value of VISUAL_it
weights = varPower()

# delta = 1, no strata 
# sigma_it = sigma * lambda_it 
#          = sigma * lambda(mu_it) 
#          = sigma * mu_it               
# NB. the scale parameter can be interpreted as a coefficient of variation, 
# constant for all timepoints.
weights = varPower(fixed = 1)              # <mu>-group



alpha = 0.05
intervals(fm9.0, which="coef", level = 1 - alpha)    # 1 - alpha/length(lm9.0$coefficients) if Bonferroni correction
intervals(fm9.0, which="var-cov", level = 1 - alpha)

In [ ]:
#### LMs with fixed effects and correlated residual errors

(Vg1 <- Variogram(fm9.2, form = ~ time | subject)) # variable | Strata

fm_fit <- gls(lm1.form, weights =                      , 
                correlation =                     ,
                data =                      )
intervals(fm12.1, which = "var-cov")   # CIs for rho, delta, sigma 
fm12.2vcov <- getVarCov(fm12.2, individual = "2") # Ri for the 2nd subject
print(cov2cor(fm12.1vcov), digits = 2, corr = TRUE, stdevs = FALSE)     # C_i



#### 1) Compound-Symmetry Correlation Structure - corCompSymm (Chapter 12.3) ####
correlation = corCompSymm(form = ~1|subject)

#### 2) Heteroscedastic Autoregressive Residual Errors - corAR1 (Chapter 12.4) ####
correlation = corAR1(form = ~tp|subject),  # NB: remember that form = ~1 | subject) would lead to a mistake

#### 3) General correlation matrix for Residual Errors - corSymm (Chapter 12.5) ####
correlation = corSymm(form = ~tp|subject)

In [ ]:
#####   LINEAR MIXED MODELS 
## 1. 'lme4' with lmer() --> it does not handle heteroscedastic residuals
## 2. 'nlme' with lme()  --> it handles heteroscedastic residuals

# •with  ́getVarCov (model, type = ’conditional’) ́we extract σ2Ri;
getVarCov(fm16.1,                     
            type = "conditional"      # sigma^2 * R_i 
            ,individual = "2")   
# Conditioned to the random effects b_i --> var-cov of the errors are independent and homoscedastic

# •with  ́getVarCov (model, type = ’marginal’) ́we extract σ2Vi;
#     sigma^2 * V_i for the second subject (type='marginal')
getVarCov(fm16.1,                       
          type = "marginal",      # sigma^2 * V_i: sigma^2*d11 extra-diagonal and sigma^2+d11 on main diagonal
          individual = "2")

# •with  ́VarCorr (model) we extract σ2D (also from the summary).
VarCorr(fm16.3)  




### homoscedastic residuals  ###   HETEROSCEDASTIC RESIDUALS (VarPower())
lm2.form <- formula(visual ~ visual0 + time + treat.f + treat.f:time)
fm16.1 <- lme(lm2.form, random =  , weights= , data = armd)
#   1. Linear Models with random intercept 
#Random intercept - Var(VISUALit) = σ2(d11 + 1)
random = ~1|subject

#   2. Linear Models with random intercept + slope 
#      2.A general structure of D
#General D - Var(VISUALit) = σ2(d11 + 2d12TIMEit + d22TIME2it + 1)
random = ~1 + time | subject

#      2.B diagonal D
#Diagonal D - Var(VISUALit) = σ2(d11 + d22TIME2it + 1)
random = list(subject = pdDiag(~time))




# Var-Cov matrix of fixed-effects
vcovb <- vcov(fm16.1) 
vcovb
# and Correlation of fixed effects
corb <- cov2cor(vcovb) 
nms <- abbreviate(names(fixef(fm16.1)), 5)
rownames(corb) <- nms
corb

# PVRE 
#------#
# i.e. the Percentage of Variance explained by the Random Effect (PVRE).
vc <- VarCorr(fm16.1), comp = c("Variance", "Std.Dev.")
var_b = as.numeric(vc[1,1]) ##### it's must be caluculted each time
var_eps = as.numeric(vc[2,1])
sd_eps <- summary(fm16.1)$sigma

PVRE <- var_b/(var_b+var_eps)
PVRE # it is high!


# Diagnostic plots 
#----------------#
# 1) Assessing Assumption on the within-group errors
plot(fm16.1)  # Pearson and raw residuals are the same now 
# (no scale is applied since we are dealing with homogeneous variance)
par(bg = "white")
qqnorm(resid(fm16.1)) # normality of the residuals
qqline(resid(fm16.1), col='red', lwd=2)

# 2) Assessing Assumption on the Random Effects
qqnorm(fm16.1, ~ranef(.), main='Normal Q-Q Plot - Random Effects on Intercept')

Read a tab-separated .txt file
df = pd.read_csv('file.txt', sep='\t')  # Adjust 'sep' if needed (e.g., ',' for comma, '|' for pipe)

Save the DataFrame as a tab-separated .txt file
df.to_csv('output.txt', sep='\t', index=False)  # Set 'index=True' if you want to include the index

  df = pd.read_csv('file.txt', sep='\t', header=None)
- Use `index=False` when saving to avoid writing row indices unless required.

#### Spatial

In [ ]:
#### 2. Load Spatial Data ####
path = 'Guerry.shp'
guerry <- st_read(guerry_path)

#### 3. Spatial Weights ####  
# - Contiguity Based Weights: `queen_weights()`, `rook_weights()`
        # queen_weights(sf_obj, order=1, include_lower_order = False, precision_threshold = 0)        
        queen_w <- queen_weights(guerry)
        summary(queen_w)

        #rook_weights(sf_obj, order=1,include_lower_order=False, precision_threshold = 0)
        rook_w <- rook_weights(guerry)
        summary(rook_w)

# - Distance Based Weights: `distance_weights()`
        #distance_weights(geoda_obj, dist_thres, power= 1.0,  is_inverse= False, is_arc= False, is_mile= True)
        dist_thres <- min_distthreshold(guerry)
        dist_thres
        dist_w <- distance_weights(guerry, dist_thres)
        summary(dist_w)

# - K-Nearest Neighbor Weights: `knn_weights()`
        # knn_weights(gda, k, power = 1.0,is_inverse = False, is_arc = False, is_mile = True)
        knn6_w <- knn_weights(guerry, 6)
        summary(knn6_w)


#### 4 Local Indicators of Spatial Association–LISA ####
    # - Local Moran: local_moran()
    lisa <- local_moran(queen_w,  guerry['Crm_prs'])
        # - lisa_clusters(): Get the local cluster indicators returned from LISA computation.
        # - lisa_colors(): Get the cluster colors of LISA computation.
        # - lisa_labels(): Get the cluster labels of LISA computation.
        # - lisa_values(): Get the local spatial autocorrelation values returned from LISA computation.
        # - lisa_num_nbrs(): Get the number of neighbors of every observations in LISA computation.
        # - lisa_pvalues(): Get the local pseudo-p values of significance returned from LISA computation.

    # - Quantile LISA: local_quantilelisa()
    # Two input parameters, k and q, need to be specified in the function local_quantilelisa(): 
    # k is the number of quantiles (k > 2), and the q is the index of selected quantile lisa ranging from 1 to k.
    qsa <- local_quantilelisa(queen_w, crm_prp, 5, 5)
    # To get the p-values and cluster indicators of the quantile LISA computation:
    lisa_pvalues(qsa)
    lisa_clusters(qsa)

#### 6 Exploratory Spatial Data Analysis ####
plot(guerry)

# With the LISA results, we can make a local Moran cluster map:
lisa_colors <- lisa_colors(lisa)
lisa_labels <- lisa_labels(lisa)
lisa_clusters <- lisa_clusters(lisa)
plot(st_geometry(guerry), 
     col=sapply(lisa_clusters, function(x){return(lisa_colors[[x+1]])}), 
     border = "#333333", lwd=0.2)
title(main = "Local Moran Map of Crm_prs")
legend('bottomleft', legend = lisa_labels, fill = lisa_colors, border = "#eeeeee")

# To create a significance map that is associated with the local Moran map, 
go to file 5.1

In [ ]:
##### Variogram modeling ##### - isotropicity
# basic variograms available
show.vgms() 

data=read.table('/Users/amirh_jandaghian/Documents/Polimi/Y 02/Polimi - Git/AS - Applied Statistics (SECCHI PIERCESARE) [2024-25]/Lab/6_Geostatistics/fluoruro.txt')
#names(data)[3]='f'
attach(data)
coordinates(data)=c('X','Y')
head(data) # --> correctly imported

## weighted least squares fitting a variogram model to the sample variogram
## STEPS:
## 1) choose a suitable model
    # sample (empirical) variogram (binned estimator)
    svgm <- variogram(log(col) ~ 1, data, cutoff = 1000, width = 1000/lag) # with all the defaults of the function
    plot(svgm, main = 'Sample Variogram', pch=19)

    # we can take into account different directions through 'alpha':
    plot(variogram(log(zinc) ~ 1, meuse, alpha = c(0, 45, 90, 135)), pch=19)

## 2) choose suitable initial values for partial sill, range & nugget
    # try reasonable initial values (remember: vgm(sill, model, range, nugget))
    v.fit <- fit.variogram(v, vgm(1, "Sph", 800, 1))

## 3) fit the model using one of the possible fitting criteria
    # plot of the final fit
    v <- variogram(log(zinc) ~ 1, meuse)
    v.fit <- fit.variogram(v, vgm(1, "Sph", 800, 1))
    plot(v, v.fit, pch = 19) # always do this!

# REML in file 

####          SPATIAL PREDICTION & KRIGING          ####

## Prediction at a single new location 
s0.new = data.frame(x=179180, y=330100) # UTM coordinates 
coordinates(s0.new) = c('x','y')        # set the coordinates of the dataframe

# Create a gstat object setting a spherical (residual) variogram
# gstat(g.obj, id, formula, data, model, set,...)
g.tr <- gstat(formula = log(zinc) ~ 1, data = meuse, model = v.fit) # v.fit is the variogram model we use

## ordinary kriging -  spatially constant mean
    # predict(obj, grid, BLUE=FALSE)
    s0=as.data.frame(matrix(c(0.3,0.24,D.s0),1,3))
    names(s0)=c('X','Y','D')
    coordinates(s0)=c('X','Y')

    predict(g.tr, s0.new)
    # var1.pred is the predicted log(zinc) 
    # var1.var variance of prediction error (ordinary kriging variance)
    # variance > 0 (as expcted)

    # this gives the estimate (best linear unbiased estimator) of the mean(trend component) under gls model
    predict(g.tr, s0.new, BLUE = TRUE)

    # this gives the estimate of the mean (drift component) under gls
    predict(g.tr, meuse[1,], BLUE = TRUE) # same mean everywhere (I am under stationarity, same as before)


    # prediction over the entire grid
    lz.ok <- predict(g.tr, meuse.grid, BLUE = FALSE)
    spplot(lz.ok[1], main='Ordinary Kriging - Prediction')
    spplot(lz.ok[2], main='Ordinary Kriging - Variance')


## universal kriging
        # Create a gstat object setting a spherical (residual) variogram
        # gstat(g.obj, id, formula, data, model, set,...)
        meuse.gstat <- gstat(id = 'zinc', formula = log(zinc) ~ sqrt(dist), # non stationary formula (intercept is automatically inside)
                            data = meuse, nmax = 50, model = v.fit, set = list(gls=1))
        # Estimate the variogram from GLS residuals:
        #?variogram.gstat
        v.gls <- variogram(meuse.gstat) # difference wrt before (before we were just giving a formula, now we give the non-stationary object)
        plot(v.gls) 
        # before it was 0.6, now 0.25
        v.gls.fit <- fit.variogram(v.gls, vgm(0.25, "Sph", 800, 0.8)) # vgm(sill, model, range, nugget)
        v.gls.fit
        plot(v.gls, v.gls.fit, pch = 19)
        # Update gstat object with variogram model
        meuse.gstat <- gstat(id = 'zinc', formula = log(zinc) ~ sqrt(dist),
                            data = meuse, nmax = 50, model = v.gls.fit, set = list(gls=1))
        meuse.gstat

    predict(meuse.gstat, s0.new) # you need to give object and new location

#### HMM

In [ ]:
# The probabilities of the fair die are (1/6, ..., 1/6) for throwing ("1",...,"6").
# The probabilities of the loaded die are (1/10, ..., 1/10, 1/2) for throwing ("1",...,"5","6"). 
# The observer doesn’t know which die is actually taken (the state is hidden), 
# but the sequence of throws (observations) can be used to infer which die (state) was used.
# Can we detect which die is in use at any given time, just by observing the sequence of rolls?
# States = {Fair, Loaded}
# P(Fair --> Fair) = 0.95
# P(Fair --> Loaded) = 0.05
# P(Loaded --> Fair) = 0.1
# P(Loaded --> Loaded) = 0.9

par(bg = "white")

States = c('Fair', 'Loaded')
Symbols = c(1, 2, 3, 4, 5, 6)
observations = c(3, 2, 2, 1, 2, 3, 6, 6, 6, 6)
transProbs = matrix( c(0.95, 0.1, 
                       0.05,  0.9), 2)
transProbs
emissProbs = matrix( c(1/6, 1/10, 
                       1/6, 1/10,
                       1/6, 1/10,
                       1/6, 1/10,
                       1/6, 1/10,
                       1/6, 1/2 ), 2)
emissProbs
hmm = initHMM(States, Symbols, c(0.5, 0.5), transProbs, emissProbs)
hmm
viterbi(hmm, observations) # we get the most probable path of hidden states


Exam



In [3]:
# System Prompt:
You are a skilled statistics student answering an exam in R. Be concise and clear. Focus on solving only what is asked. For each task:
- Use clean R code with minimal comments (only where necessary).
- Avoid verbose explanations or unnatural phrasing.
- Final answers should be highlighted with a short comment (e.g., `# FINAL ANSWER:`).
- No over-explaining, just what a capable student would write under time pressure.
- write all codes in a block and seperate each question with a comment.

Question: 

Data:

Warning message in file(file, "rt"):
"cannot open file 'data.text': No such file or directory"


ERROR: Error in file(file, "rt"): cannot open the connection


In [ ]:

Question: 

Data: